In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [32]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [33]:
df = fetch_hourly_rides(12)

2025-03-04 23:18:17,875 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 23:18:17,908 INFO: Initializing external client
2025-03-04 23:18:17,908 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 23:18:18,792 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215687
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.08s) 


In [34]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-04 14:00:00+00:00,186,164
1,2025-03-04 13:00:00+00:00,154,0
2,2025-03-04 18:00:00+00:00,62,0
3,2025-03-04 11:00:00+00:00,72,1
4,2025-03-04 17:00:00+00:00,212,0
...,...,...,...
3007,2025-03-04 11:00:00+00:00,12,2
3008,2025-03-04 15:00:00+00:00,183,0
3009,2025-03-04 17:00:00+00:00,52,0
3010,2025-03-04 14:00:00+00:00,194,0


In [35]:
df_pred = fetch_predictions(240)

2025-03-04 23:18:32,834 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 23:18:32,847 INFO: Initializing external client
2025-03-04 23:18:32,850 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 23:18:33,685 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215687
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 


In [36]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,195,0.0,2025-03-04 22:00:00+00:00
1,93,0.0,2025-03-04 22:00:00+00:00
2,90,76.0,2025-03-04 22:00:00+00:00
3,213,0.0,2025-03-04 22:00:00+00:00
4,258,0.0,2025-03-04 22:00:00+00:00
...,...,...,...
497,155,0.0,2025-03-05 00:00:00+00:00
498,189,0.0,2025-03-05 00:00:00+00:00
499,107,56.0,2025-03-05 00:00:00+00:00
500,88,14.0,2025-03-05 00:00:00+00:00


In [37]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [38]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
0,2025-03-04 22:00:00+00:00,59,0,0.0
1,2025-03-04 22:00:00+00:00,217,1,1.0
2,2025-03-04 22:00:00+00:00,62,0,0.0
3,2025-03-04 22:00:00+00:00,220,0,0.0
4,2025-03-04 22:00:00+00:00,77,0,0.0
...,...,...,...,...
246,2025-03-04 22:00:00+00:00,7,0,1.0
247,2025-03-04 22:00:00+00:00,52,0,1.0
248,2025-03-04 22:00:00+00:00,251,0,0.0
249,2025-03-04 22:00:00+00:00,11,0,0.0


In [39]:
pd.to_datetime(df['pickup_hour']).dt.floor('h')

0      2025-03-04 14:00:00+00:00
1      2025-03-04 13:00:00+00:00
2      2025-03-04 18:00:00+00:00
3      2025-03-04 11:00:00+00:00
4      2025-03-04 17:00:00+00:00
                  ...           
3007   2025-03-04 11:00:00+00:00
3008   2025-03-04 15:00:00+00:00
3009   2025-03-04 17:00:00+00:00
3010   2025-03-04 14:00:00+00:00
3011   2025-03-04 16:00:00+00:00
Name: pickup_hour, Length: 3012, dtype: datetime64[us, Etc/UTC]

In [40]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [41]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
18,2025-03-04 22:00:00+00:00,2,0,0.0,0.0
131,2025-03-04 22:00:00+00:00,3,0,0.0,0.0
60,2025-03-04 22:00:00+00:00,4,3,16.0,13.0
69,2025-03-04 22:00:00+00:00,6,0,0.0,0.0
246,2025-03-04 22:00:00+00:00,7,0,1.0,1.0
...,...,...,...,...,...
176,2025-03-04 22:00:00+00:00,259,0,0.0,0.0
134,2025-03-04 22:00:00+00:00,260,0,1.0,1.0
13,2025-03-04 22:00:00+00:00,261,22,19.0,-3.0
169,2025-03-04 22:00:00+00:00,262,10,25.0,15.0


In [42]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
0,2025-03-04 22:00:00+00:00,59,0,0.0,0.0
1,2025-03-04 22:00:00+00:00,217,1,1.0,0.0
2,2025-03-04 22:00:00+00:00,62,0,0.0,0.0
3,2025-03-04 22:00:00+00:00,220,0,0.0,0.0
4,2025-03-04 22:00:00+00:00,77,0,0.0,0.0
...,...,...,...,...,...
246,2025-03-04 22:00:00+00:00,7,0,1.0,1.0
247,2025-03-04 22:00:00+00:00,52,0,1.0,1.0
248,2025-03-04 22:00:00+00:00,251,0,0.0,0.0
249,2025-03-04 22:00:00+00:00,11,0,0.0,0.0


In [43]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

df['pickup_hour'] = pd.to_datetime(df['pickup_hour']).dt.floor('h')
df_pred['pickup_hour'] = pd.to_datetime(df_pred['pickup_hour']).dt.floor('h')


# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [44]:
mae_by_hour["MAE"].mean()

11.593625498007968